In [1]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [2]:
import numpy as np
import pandas as pd

df1 = pd.read_csv('Cleaned-P&P data.csv')

blanks = []  # start with an empty list`

for i,c,d,inc,sd in df1.itertuples():  # iterate over the DataFrame
        if c == 'O' or c=='E':         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
df1.drop(blanks, inplace=True)
df1["desc"] = df1["short_des"] + '. ' + df1["desc"]
df1.reset_index(drop=True,inplace=True)


In [3]:

df2 = pd.read_csv('Cleaned-P&P data.csv')
blanks2 = []  # start with an empty list`

for i,c,d,inc,sd in df2.itertuples():  # iterate over the DataFrame
        if c == 'O' or c=='E':         # test 'review' for whitespace
            blanks2.append(i)     # add matching index numbers to the list
df2.drop(blanks2, inplace=True)
df2.reset_index(drop=True,inplace=True)

In [4]:
dataset = pd.concat([df1, df2], ignore_index=True)

In [5]:
# Cleaning the texts
import re
corpus = []
all_words = []
max_len=0
for i in range(0, 490):

    review = re.sub('\w\d{7}', ' ', dataset['desc'][i])
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    
    review = nlp(review)
    review = [word.text for word in review]
   # for word.text in review:
        
    all_words.append(review) 
    if len(review) > max_len:
        max_len = len(review)
    ds = ' '.join(review)
#    for word2vec we want an array of vectors
    corpus.append(ds)
    

### Standardise key terms in data

In [6]:
dpf = pd.read_csv('Del-Pass.csv')



In [7]:
dpf['kpattern']

0     delivery subscription service
1        delivery pass subscription
2             delivery pass service
3                free delivery pass
4                   delivery passes
5                    delivery pass 
6        p&p delivery subscription 
7            delivery subscriptions
8             delivery subscription
9                      delivery sub
10                      p&p service
11                              p&p
12                     subscription
Name: kpattern, dtype: object

In [8]:
keys_list = dpf['kpattern'].tolist()
regex = re.compile(r'(' + '|'.join(keys_list) + r')') 

In [9]:
cus_list = ['remove','removed','cancelled', 'cancel' ,'deleted']
regey = re.compile(r'(' + '|'.join(cus_list) + r')') 

In [10]:
import nltk
from nltk.corpus import stopwords
stopwords=nltk.corpus.stopwords.words('english')
additional =["thank you","please","hello","hi","advise","hin't","st","nd","rd","th","thank","pnt","work request"]
stopwords = stopwords + additional
remov=[]
for ele in stopwords:
    matches = re.findall("n't",ele)
    matches2 = re.findall("'nt",ele)
    if len(matches)>0 or len(matches2)>0:
        remov.append(ele)
stopwords = [word for word in stopwords if word not in remov]

In [11]:
cldict = {'P':1 , 'R':0}
dataset['Class Label'] = dataset['class'].map(cldict)
label_series = pd.Series(dataset['Class Label'])

In [12]:
from keras.utils import to_categorical

class_label = np.array(label_series)
y = class_label
y.shape

Using TensorFlow backend.


(490,)

In [14]:
dataset.head(5)

,class,desc,inc_num,short_des,Class Label
0,P,"y2684992. good afternoon,\r\n\r\nthis customer...",INC0809260,y2684992,1
1,R,delivery pass needs removing on staff accounts...,INC0808628,delivery pass needs removing on staff accounts...,0
2,R,delivery pass needs removing on staff accounts...,INC0807253,delivery pass needs removing on staff accounts...,0
3,P,customer being charged for delivery. customer:...,INC0806836,customer being charged for delivery,1
4,P,website/web shop issues. short description:tam...,INC0805994,website/web shop issues,1


In [13]:
X = corpus

In [14]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=47)


### Create custom rules 

In [18]:
#segment_featurizer = SegmentFeaturizer()  # more on this below
class CustomLinguisticFeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass   
  #  def fit(self, X, y=None):
    def fit(self, X, y):
        return self    
    def transform(self, X):       
        ref_corpus=[]
        for text in X:           
            trans = regex.sub(lambda m: m.group().replace(m.group(),"del-pass"),text)
            trans_ref = regey.sub(lambda n: n.group().replace(n.group(),"delete"),trans)
             
            ref_corpus.append(trans_ref)
       
        return ref_corpus

### Spacy Vector Transformer

In [17]:

from sklearn.base import BaseEstimator, TransformerMixin
class SpacyVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp
        self.dim = 300
        pass
    def fit(self, X, y):
        return self

    def transform(self, X):
        # Doc.vector defaults to an average of the token vectors.
        # https://spacy.io/api/doc#vector
        lemmatized_list=[]
        for snt in X:
        #    print(snt)
            tokens = nlp(snt)
        #    for token in tokenized:
            filtered_sentence = [w.text for w in tokens if not w.text in stopwords]
        #                 lemm = token.lemma_ for token.text in token
        #             lemmatized_list.append(lemm)
            stri = ' '.join(filtered_sentence)
            lemmatized_list.append(stri)
        return [self.nlp(text).vector for text in lemmatized_list]
       


### Reduce dimensionality

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

embeddings_pipeline = Pipeline(
    steps=[
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(50)),
        ("classifier", RandomForestClassifier()),
    ]
)
embeddings_pipeline.fit(X_train, y_train)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

embeddings_pipeline2 = Pipeline(
    steps=[
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(50)),
        ("classifier", LinearSVC()),
    ]
)
embeddings_pipeline2.fit(X_train, y_train)

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
svm = LinearSVC()
embeddings_pipeline3 = Pipeline(
    steps=[
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(50)),
        ("classifier", CalibratedClassifierCV(svm)),
    ]
)
embeddings_pipeline3.fit(X_train, y_train)

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_extraction.text import TfidfVectorizer
svm = LinearSVC()
embeddings_pipeline4 = Pipeline(
    steps=[
        ("cust",CustomLinguisticFeatureTransformer()),
#        ("tfidf", TfidfVectorizer()),
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(200)),
#         ("dict_vect", DictVectorizer()),
        ("classifier", CalibratedClassifierCV(svm)),
    ]
)
embeddings_pipeline4.fit(X_train, y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
#svm = LinearSVC()
embeddings_pipeline5 = Pipeline(
    steps=[
        ("cust",CustomLinguisticFeatureTransformer()),
        ("tfidf", TfidfVectorizer()),
#        ("mean_embeddings", SpacyVectorTransformer(nlp)),
#        ("reduce_dim", TruncatedSVD(200)),
#         ("dict_vect", DictVectorizer()),
#        ("rdf", RandomForestClassifier()),
        ("mnb",MultinomialNB()),
    ]
)
embeddings_pipeline5.fit(X_train, y_train)

In [ ]:

y_pred = embeddings_pipeline4.predict(X_test)
cr = classification_report(y_test, y_pred)

In [ ]:
print(cr)

In [ ]:
#list1 = ['issue with delivery subscriptions Piara called in and reported an issue that one of her customer purchased delivery subscriptions in month of january for year costs pounds but still got charged for delivery kindly assist ']
#list1 =['delivery subscription not active on account please can you activate delivery subscription rv (purchased ) on account thanks ']
#list1 = ['delivery pass needs removing on staff accounts orders supra dba req please can the delivery pass be removed from the attached staff accounts thanks ']
list1 = ['application session restarted before raising incidents customer unable to edit the address for the same application session restarted before raising incidents customer also unable to edit the address for the same']
print(embeddings_pipeline4.predict(list1))
probs = embeddings_pipeline4.predict_proba(list1)
np.around(probs, decimals = 3)*100

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(embeddings_pipeline5, X, y, cv = 10)

In [ ]:
cv_mean_score = np.mean(cross_val_score(embeddings_pipeline3, X, y, cv = 10))
clf_score = embeddings_pipeline3.score(X_test, y_test)
cv_mean_score, clf_score

In [ ]:
predictions = embeddings_pipeline5.predict(X)
len(X)

In [ ]:
clf_score = embeddings_pipeline3.score(X, y)
clf_score

In [ ]:
Faulty_Xtest =[]
for i in range(0,490):
    if y[i] != predictions[i]:
        print(f'predicted class of:{X[i]} is :{predictions[i]}', f'Index is : {i}')
        Faulty_Xtest.append(X[i])
arr = np.array(Faulty_Xtest)
arr

In [ ]:
dataset['inc_num'][201], dataset['desc'][201], dataset['class'][201]

In [ ]:
# Enter your code here:

for token in nlp(X_test[94]):
    print(f'{token.text:{12}} {token.pos_:{6}} {token.dep_:{6}} {spacy.explain(token.dep_)}')

In [ ]:

from spacy import displacy
displacy.render(nlp(X_test[95]), style='ent', jupyter=True)

In [ ]:
import re
for match in re.finditer("delivery subscription ",X_test[94]):
    X_test[94]
    print(match.span())


In [ ]:
lemmatized_list=[]
for snt in X[:5]:
    print(snt)
    tokens = nlp(snt)
#    for token in tokenized:
    filtered_sentence = [w.text for w in tokens if not w.text in stopwords]
#                 lemm = token.lemma_ for token.text in token
#             lemmatized_list.append(lemm)
    stri = ' '.join(filtered_sentence)
    lemmatized_list.append(stri)
lemmatized_list

In [ ]:
matche = re.findall("delivery subscription ",X_test[94])
match.span()

In [ ]:
regex = re.compile('([a-zA-Z]\"[a-zA-Z])', re.S)
myfile =  'foo"s bar'
myfile2 = regex.sub(lambda m: m.group().replace('"',"%",1), myfile)
print(myfile2)

In [ ]:


lst = ['delivery subscription service', 'delivery pass subscription','delivery pass service','delivery passes','delivery pass ','p&p delivery subscription ','delivery subscriptions','delivery subscription']

def replace_double_quote(match):
    text = match.group()
    print(match.group())
    return text.replace(text, 'del-pass')
regex = re.compile(r'(' + '|'.join(lst) + r')')
myfile = 'please can you apply the annual delivery subscription is activated on this customers account purchased still being charged for delivery thank you'
regex.sub(replace_double_quote, myfile)



In [ ]:


lst = ['foo', 'bar', 'and']
def my_func():
    
    regex = re.compile(r'(' + '|'.join(lst) + r')')
    myfile = 'foo"s bar and bar"s foo'
   
    pxk = regex.sub(lambda m: m.group().replace(m.group(),"rap",1), myfile)
    return pxk
#     def replace_double_quote(match):
#         text = match.group()
#         print(text)
#         return text.replace(text, 'rap')
   
print(my_func())

In [ ]:

def match_kpatterns(match):
    
    text = match.group()
    print(match.group())
    return text.replace(text, 'del-pass')


regez = re.compile(r'(' + '|'.join(keys_list) + r')')
seg = "please can you apply the annual delivery subscription is activated on this customers account purchased still being charged for delivery thank you"

regez.sub(match_kpatterns, seg)
#    return regex.sub(lambda m: m.group().replace(m.group(),"del-pass"), rfseg)
#r_featurize(seg)

In [ ]:
text = nlp(u'trashbin')
text1 = nlp(u'dustbin')
fr = text1.similarity(text)
fr

In [ ]:
for i in range(0,488):
    matches = re.findall("n t ",corpus[i])
    print(f'mathc is : {matches}', i)

In [ ]:
lemm=[]
#for snt in corpus[0]:
kr = nlp(corpus[0])
for token in kr:
    ls = token.lemma_
    lemm.append(ls)
lemm

In [ ]:
nlp(lemm[0]).vector

In [ ]:

from joblib import dump, load
dump(embeddings_pipeline4, filename = 'ii' )

In [ ]:
from sklearn.decomposition import NMF
nmf_model = NMF(n_components=3,random_state=42)
# This can take awhile, we're dealing with a large amount of documents!
nmf_model.fit()

In [19]:
from joblib import load
loaded_model2 = load(filename = 'P&P Classifier V_0.1.joblib')

In [22]:
from sklearn.metrics import classification_report
y_pred = loaded_model2.predict(X)
cr = classification_report(y, y_pred)

In [23]:
print(cr)
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y,y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       190
           1       1.00      1.00      1.00       300

   micro avg       1.00      1.00      1.00       490
   macro avg       1.00      1.00      1.00       490
weighted avg       1.00      1.00      1.00       490



array([[190,   0],
       [  0, 300]], dtype=int64)